In [ ]:
# Takes the output from routing_3.ipynb and works on the routing portion of it
import polars as pl
import numpy as np
from math import pi, cos, sin, sqrt, atan2
import networkx as nx 
import random
import os
import rtsvg
rt = rtsvg.RACETrack()

file_no = 7
df_collapsed    = pl.read_parquet(f'../../data/stanford/facebook/348.edges_collapsed_edges.{file_no}.parquet')
df_edge_arc_pos = pl.read_parquet(f'../../data/stanford/facebook/348.edges_arc_pos.{file_no}.parquet')
df_collapsed.sample(3)

In [ ]:
df_edge_arc_pos.sample(3)

In [3]:
def extractCircles(_df_):
    circles, circle_lu = [], {}
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        circle_lu[k] = len(circles)
        circles.append(k)
    return circles, circle_lu

def validatePositions(_df_, clearance=10): # edge_arc_pos
    # Extract the circles first
    circles, circle_lu = extractCircles(_df_)
    # Then make sure the positions don't fall within a circle & that they have some clearance
    for k, k_df in _df_.group_by(['x','y', 'x_circle', 'y_circle', 'r_circle']):
        _xy_     = (k[0], k[1])
        _uv_     = rt.unitVector((k[2:4], _xy_))
        _xy_end_ = (_xy_[0]+ clearance*_uv_[0], _xy_[1]+ clearance*_uv_[1])
        _my_circle_ = k[2:5]
        for c in circles:
            if c == _my_circle_: continue
            _length_ = rt.segmentLength((_xy_,c))
            if _length_ < c[2] + clearance:
                raise Exception(f'node key "{k_df["node_key"]}" with radius circle {c}')
            _length_ = rt.segmentLength((_xy_end_, c))

def extents(_df_): # edge_arc_pos -- assumes that the coordinates are already in screen space
    xmin, ymin, xmax, ymax = 1e9, 1e9, -1e9, -1e9
    for k, k_df in _df_.group_by(['x_circle','y_circle','r_circle']):
        xmin, ymin = min(xmin, k[0]-k[2]), min(ymin, k[1]-k[2])
        xmax, ymax = max(xmax, k[0]+k[2]), max(ymax, k[1]+k[2])
    for k, k_df in _df_.group_by(['x','y']):
        xmin, ymin = min(xmin, k[0]), min(ymin, k[1])
        xmax, ymax = max(xmax, k[0]), max(ymax, k[1])
    return xmin, ymin, xmax, ymax

validatePositions(df_edge_arc_pos)
_ext_ = extents(df_edge_arc_pos)

In [4]:
w, h, x_ins, y_ins, border = 1000, 1000, 30.0, 30.0, 20.0
x_canvas, y_canvas, w_canvas, h_canvas = _ext_[0]-x_ins, _ext_[1]-y_ins, 2*x_ins+_ext_[2]-_ext_[0], 2*y_ins+_ext_[3]-_ext_[1]
# Base Circles
svg_hdr =     [f'<svg  x="0" y="0" width="{w}" height="{h}" viewBox="{x_canvas} {y_canvas} {w_canvas} {h_canvas}">']
svg_hdr.append(f'<rect x="{x_canvas}" y="{y_canvas}" width="{w_canvas}" height="{h_canvas}" fill="#ffffff" />')
svg_hdr = ''.join(svg_hdr)
svg_ftr = '</svg>'
svg_base = []
circles, circles_lu = extractCircles(df_edge_arc_pos)
for i in range(len(circles)):
    cx, cy, r = circles[i]
    svg_base.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="3" />')

# LaGuerre Voronoi
_box_ = [(_ext_[0]-border,_ext_[1]-border),(_ext_[0]-border,_ext_[3]+border),(_ext_[2]+border,_ext_[3]+border),(_ext_[2]+border,_ext_[1]-border)]
voronoi_polys = rt.laguerreVoronoi(circles, Box=_box_)
svg_voronoi   = []
for i in range(len(voronoi_polys)):
    d = f'M {voronoi_polys[i][0][0]} {voronoi_polys[i][0][1]} '
    for j in range(1, len(voronoi_polys[i])): d += f'L {voronoi_polys[i][j][0]} {voronoi_polys[i][j][1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{rt.co_mgr.getColor(i)}" stroke-width="1"/>')
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="#000000" stroke-width="0.1"/>')
_seen_ = set()
svg_voronoi_circle_markers = []
for i in range(len(voronoi_polys)):
    for j in range(len(voronoi_polys[i])):
        x, y = voronoi_polys[i][j]
        if (x,y) not in _seen_: svg_voronoi_circle_markers.append(f'<circle cx="{x}" cy="{y}" r="{4.0+random.random()*4.0}" stroke-width="0.1" stroke="#000000" fill="none"/>')
        _seen_.add((x,y))

# (Uniquify) Segment To Poly
def uniquifySegment(s):
    _xy0_, _xy1_ = s[0], s[1]
    if   _xy0_[0] <  _xy1_[0]: return s
    elif _xy0_[0] >  _xy1_[0]: return (_xy1_, _xy0_)
    elif _xy0_[1] <  _xy1_[1]: return s
    else:                      return (_xy1_, _xy0_)

#rt.tile([svg_hdr + ''.join(svg_base) + ''.join(svg_voronoi_circle_markers) + svg_ftr])

In [5]:
segment_to_poly_is = {}
for i in range(len(voronoi_polys)):
    _poly_ = voronoi_polys[i]
    for j in range(len(_poly_)):
        _segment_    = (_poly_[j], _poly_[(j+1)%len(_poly_)])
        _uniquified_ = uniquifySegment(_segment_)
        if _uniquified_ not in segment_to_poly_is: segment_to_poly_is[_uniquified_] = []
        segment_to_poly_is[_uniquified_].append(i)

def addVertexAndUpdatePolygons(seg, xy, pixel_diff=4.0):
    seg    = uniquifySegment(seg)
    l      = rt.segmentLength(seg)
    l0, l1 = rt.segmentLength((seg[0], xy)), rt.segmentLength((seg[1], xy))
    if l0 < pixel_diff: return seg[0]
    if l1 < pixel_diff: return seg[1]
    polys_to_update = segment_to_poly_is[seg]
    for i in polys_to_update:
        _poly_ = voronoi_polys[i]
        j      = 0
        while _poly_[j] != seg[0] and _poly_[j] != seg[1]: j += 1
        k      = (j+1)%len(_poly_)
        if    _poly_[k] != seg[0] and _poly_[k] != seg[1]: k = len(_poly_)-1
        if    _poly_[k] != seg[0] and _poly_[k] != seg[1]: raise Exception('can\'t find k')
        if j == 0 and k == len(_poly_)-1: _poly_.append(xy)
        else:                             _poly_.insert(k,xy)
    del segment_to_poly_is[seg]
    seg0, seg1 = uniquifySegment((xy,seg[0])), uniquifySegment((xy,seg[1]))

    if seg0 not in segment_to_poly_is: segment_to_poly_is[seg0] = []
    if seg1 not in segment_to_poly_is: segment_to_poly_is[seg1] = []
    segment_to_poly_is[seg0].extend(polys_to_update)
    segment_to_poly_is[seg1].extend(polys_to_update)
    return xy

# Entry / Exit Points
_large_distance_ = 1e9
svg_pts, _edges_, _pos_, _pos_vertex_names_ = [], [], {}, {}
def getOrCreateVertexName(xy):
    if xy not in _pos_vertex_names_: _pos_vertex_names_[xy] = f'V_{len(_pos_vertex_names_)}'
    return _pos_vertex_names_[xy]
for k, k_df in df_edge_arc_pos.group_by(['node_key', 'x', 'y', 'x_circle', 'y_circle', 'r_circle']):
    _xy_, _cxy_, _circle_ = k[1:3], k[3:5], k[3:6]
    circle_i              = circles_lu[_circle_] # Validate circle exists
    _uv_                  = rt.unitVector((_cxy_, _xy_))
    if  k[0].endswith('_to_'): exit_pt, _color_ = False, '#ff0000' # Entry ... where something stops  (enters the circle)
    else:                      exit_pt, _color_ = True,  '#0000ff' # Exit  ... where something starts (leaves the circle)
    svg_pts.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="2" fill="{_color_}" stroke="none" />')
    # Determine where the entry/exit point connects to the voronoi segments
    entry_exit_ray = (_xy_, (_xy_[0] + _large_distance_*_uv_[0], _xy_[1] + _large_distance_*_uv_[1]))
    xy_inter, _segment_ = None, None
    _poly_ = voronoi_polys[circle_i]
    for i in range(len(_poly_)):
        _segment_          = (_poly_[i], _poly_[(i+1)%len(_poly_)])
        _intersects_tuple_ = rt.segmentsIntersect(_segment_, entry_exit_ray)
        if _intersects_tuple_[0]:
            xy_inter = (_intersects_tuple_[1], _intersects_tuple_[2])
            break
        xy_inter = None
    if xy_inter is None: raise Exception('xy_inter should not be none...')
    xy_inter = addVertexAndUpdatePolygons(_segment_, xy_inter) # possibility that maybe the new vertex isn't that different from an existing vertex
    _edges_.append((k[0], getOrCreateVertexName(xy_inter)))
    _pos_[k[0]]                            = _xy_
    _pos_[getOrCreateVertexName(xy_inter)] = xy_inter
    svg_pts.append(f'<line x1="{_xy_[0]}" y1="{_xy_[1]}" x2="{xy_inter[0]}" y2="{xy_inter[1]}" stroke="{_color_}" stroke-width="0.4"/>')

# Add the updated polys as edges to the edge list
already_added = set()
for i in range(len(voronoi_polys)):
    _poly_ = voronoi_polys[i]
    for j in range(len(_poly_)):
        _xy0_, _xy1_ = _poly_[j], _poly_[(j+1)%len(_poly_)]
        _seg_ = uniquifySegment((_xy0_, _xy1_))
        if _seg_ not in already_added:
            _edges_.append((getOrCreateVertexName(_seg_[0]), getOrCreateVertexName(_seg_[1])))
            _pos_[getOrCreateVertexName(_seg_[0])], _pos_[getOrCreateVertexName(_seg_[1])] = _seg_[0], _seg_[1]
            already_added.add(_seg_)

#rt.tile([svg_hdr + ''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi) + svg_ftr])

In [ ]:
svg_voronoi_connects, xys_seen = [], set()
for i in range(len(voronoi_polys)):
    _poly_ = voronoi_polys[i]
    for j in range(len(_poly_)):
        _xy0_, _xy1_ = _poly_[j], _poly_[(j+1)%len(_poly_)]
        xys_seen.add(_xy0_), xys_seen.add(_xy1_)
        svg_voronoi_connects.append(f'<line x1=\"{_xy0_[0]}\" y1=\"{_xy0_[1]}\" x2=\"{_xy1_[0]}\" y2=\"{_xy1_[1]}\" stroke=\"{rt.co_mgr.getColor(i)}\" stroke-width=\"0.5\"/>')
for _xy_ in xys_seen: svg_voronoi_connects.append(f'<circle cx=\"{_xy_[0]}\" cy=\"{_xy_[1]}\" r=\"{3.0 + random.random()}\" fill=\"none\" stroke=\"#000000\" stroke-width=\"0.1\"/>')
rt.tile([svg_hdr + ''.join(svg_base)+''.join(svg_pts)+''.join(svg_voronoi_connects) + svg_ftr])

In [ ]:
relates, g_lu = [('__fm__','__to__')], {'__fm__':[], '__to__':[],'__dist__':[]}
for _edge_ in _edges_: 
    g_lu['__fm__'].append(_edge_[0])
    g_lu['__to__'].append(_edge_[1])
    xy0, xy1 = _pos_[_edge_[0]], _pos_[_edge_[1]]
    g_lu['__dist__'].append(rt.segmentLength((xy0,xy1)))
df_g = pl.DataFrame(g_lu)
df_g = df_g.with_columns((1.0/(pl.col('__dist__')+0.1)).alias('inv_dist')) # invert it to match networkx's version of closeness
g    = rt.createNetworkXGraph(df_g, relates, count_by='inv_dist')
rt.tile([rt.link(df_g, relates, _pos_, node_size='small', count_by='__dist__', 
                 link_size='vary', link_size_max=8.0, link_size_min=1.0, w=680, h=680),
         rt.histogram(df_collapsed, bin_by='__to__', count_by='__fm__', w=256, h=680)], spacer=20)

In [ ]:
# For all "To"'s
_tiles_, _edge_frequency_ = [], {}
for k, k_df in df_collapsed.group_by(['__to__']):
    _to_ = k[0]
    # Calculate the shortest path to all other nodes
    _weights_lu_, _paths_lu_ = nx.shortest_paths.single_source_bellman_ford(g, source=_to_, weight='weight')
    # Get the list of "From"'s for this particular "To"
    fms = set(df_collapsed.filter(pl.col('__to__') == _to_)['__fm__'])
    # Create a subgraph of all the shortest paths and add it to a tile
    fmto_lu = {'__fm__':[], '__to__':[]}
    _edge_already_seen_ = set() # already seen filter is reset per every "To"
    for _fm_ in fms:
        _path_ = _paths_lu_[_fm_]
        for i in range(len(_path_)-1):
            fmto_lu['__fm__'].append(_path_[i])
            fmto_lu['__to__'].append(_path_[i+1])
            # Keep track of edge frequency counts
            if _path_[i] < _path_[i+1]: _edge_ = (_path_[i],   _path_[i+1])
            else:                       _edge_ = (_path_[i+1], _path_[i])
            if _edge_ not in _edge_frequency_:      _edge_frequency_[_edge_]  = 0
            if _edge_ not in _edge_already_seen_:   _edge_frequency_[_edge_] += 1
            _edge_already_seen_.add(_edge_)
    for _edge_ in _edge_already_seen_: g.add_edge(_edge_[0], _edge_[1], weight=g[_edge_[0]][_edge_[1]]['weight']*2.0)
    _tiles_.append(rt.link(pl.DataFrame(fmto_lu), relates, _pos_, node_size='small', w=300, h=300))

# Display the most often path edges were seen in the above algorithm
_sorter_ = []
for k in _edge_frequency_: _sorter_.append((_edge_frequency_[k], k))
_sorter_.sort(reverse=True)
for i in range(10): print(_sorter_[i])
histo_lu = {'weight':[],'tuple':[]}
for i in range(len(_sorter_)): histo_lu['weight'].append(_sorter_[i][0]), histo_lu['tuple'].append(str(_sorter_[i][1]))
rt.histogram(pl.DataFrame(histo_lu),bin_by='tuple',count_by='weight',bar_h=10,w=1024,h=512, draw_distribution=True)

In [9]:
# Display the tiles
#rt.table(_tiles_, per_row=5, spacer=10)

In [ ]:
_lu_ = {'__fm__':[], '__to__':[], '__count__':[]}
for _tuple_ in _sorter_: _lu_['__fm__'].append(_tuple_[1][0]), _lu_['__to__'].append(_tuple_[1][1]), _lu_['__count__'].append(_tuple_[0])
df_edge_usage = pl.DataFrame(_lu_)
rt.xy(df_edge_usage, x_field='__count__', y_field='__fm__')
rt.link(df_edge_usage, relates, _pos_, count_by='__count__', link_size='vary', link_size_max=8.0, link_size_min=1.0, w=800, h=800)

In [ ]:
g_example = nx.Graph()
g_example.add_edge('a','b',weight=1)
g_example.add_edge('b','c',weight=1)
g_example.add_edge('c','d',weight=1)
g_example.add_edge('d','e',weight=1)
g_example.add_edge('a','z',weight=1)
g_example.add_edge('z','e',weight=1)
print(nx.single_source_bellman_ford(g_example, 'a', weight='weight'))
g_example.add_edge('a','z',weight=10)
print(nx.single_source_bellman_ford(g_example, 'a', weight='weight')[0])
print(nx.single_source_bellman_ford(g_example, 'a', weight='weight')[1])

In [ ]:
nx.all